In [1]:
import pandas as pd
import json
import sys
import re
from bs4 import BeautifulSoup
import time
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEnginePage
import warnings
warnings.filterwarnings('ignore')

urls = pd.read_csv('./data/crunchbase_url.csv', index_col=[0])
urls['crunch_url'] = urls['url'].str.replace('https://www.crunchbase.com/organization/','')
urls

,company,url,crunch_url
0,Walmart,https://www.crunchbase.com/organization/walmart,walmart
1,Amazon,https://www.crunchbase.com/organization/amazon,amazon
2,Apple,https://www.crunchbase.com/organization/apple,apple
3,CVS Health,https://www.crunchbase.com/organization/cvs-ca...,cvs-caremark
4,UnitedHealth Group,https://www.crunchbase.com/organization/united...,unitedhealth-group
...,...,...,...
495,Hasbro,https://www.crunchbase.com/organization/hasbro...,hasbro-inc
496,Roper Technologies,https://www.crunchbase.com/organization/roper-...,roper-industries
497,ARKO,https://www.crunchbase.com/organization/arko-3d08,arko-3d08
498,Frontier Communications,https://www.crunchbase.com/organization/fronti...,frontier-communications-corporation


In [ ]:
BASE_URL = 'https://www.crunchbase.com'

companies = []
pages = []
crunchbase = {}

class Page(QWebEnginePage):
    def __init__(self, url):
        self.app = QApplication(sys.argv)
        QWebEnginePage.__init__(self)
        self.html = ''
        self.loadFinished.connect(self._on_load_finished)
        self.load(QUrl(url))
        self.app.exec_()

    def _on_load_finished(self):
        self.html = self.toHtml(self.Callable)

    def Callable(self, html_str):
        self.html = html_str
        self.app.quit()


def get_page(route):
    if not route:
        return None

    try:
        url = f'{BASE_URL}{route}'
        pages.append(Page(url))
        soup = BeautifulSoup(pages[-1].html, 'lxml')
        pages[-1].deleteLater()
    except:
        return None
    else:
        return soup


def format_name(name):
    return name.lower().replace('\n', '').strip().replace('.', '-').replace(' ', '-').replace(':', '-')


def extract_link(element):
    return re.search(r'(https?:\/\/)(www\.)?([a-zA-Z0-9]+(-?[a-zA-Z0-9])*\.)+([a-z]{2,})(\/\S*)?', element).group(0)


def print_green(s):
    print(f'\033[92m{s}\033[0m')

def company_info(i, soup):
    company = {}
    company['name'] = urls['company'][i]

    # headquaters
    try:
        elem = soup.select('field-formatter > identifier-multi-formatter > span > a')
        company['headquaters_location_city'] = elem[0].text.lstrip()
        company['headquaters_location_state'] = elem[1].text.lstrip()
        company['headquaters_location_country'] = elem[2].text.lstrip()
    except:
        print("No headquaters location")
        
    
    # Head quater reigions
    elems = soup.select('ul > li > field-formatter > identifier-multi-formatter > span > a')
    location = [elem.text.lstrip() for elem in elems if 'location_group_identifiers' in elem.get('href')]
    company['headquaters_regions'] = location
    
    # Founders
    founder = [elem.text.lstrip() for elem in elems if 'person' in elem.get('href')]
    company['founder'] = founder
    
    # website link
    try:
        elem = soup.select('ul > li > label-with-icon > span > field-formatter > link-formatter > a')[0].get('href')
        company['website_link'] = elem
    except:
        print("No website link")

    # Related Hubs 
    hub = [elem.text.lstrip() for elem in elems if 'hub' in elem.get('href')]
    company['hub'] = hub

    
    # Last Funding Type
    # Number of Exits
    elems = soup.select('label-with-icon > span > field-formatter > a')
    for elem in elems:
        if 'num_employees_enum' in elem.get('href'):
            company['num_employees_enum'] = elem.text.lstrip().rstrip()
        if 'last_funding_type' in elem.get('href'):
            company['last_funding_type'] = elem.text.lstrip().rstrip()
        if 'rank_org_company' in elem.get('href'):
            company['rank_org_company'] = elem.text.lstrip().rstrip()
    
    # IPO status
    try:
        elem = soup.select('label-with-icon > span > field-formatter > span')[0]
        company['IPO_status'] = elem.text.lstrip()
    except:
        print("No IPO status")
        
    # industries
    elem = soup.select('field-formatter > identifier-multi-formatter > span > chips-container > a > *')
    elems = [e.text.lstrip() for e in elem]
    company['industries'] = elems

    # Stock Symbol
    try:
        elem = soup.select('ul > li > field-formatter > link-formatter > a')[0]
        company['stock_symbol'] = elem.text.lstrip()
    except:
        print("No stock symbol")

    
    elems = soup.select('ul > li > field-formatter > span')
    if len(elems) == 4:
        company['founded_date'] = elems[0].text.lstrip() # Founded date
        company['operating_status'] = elems[1].text.lstrip() # Operating Status Active
        company['also_known_as'] = elems[2].text.lstrip() #Also Known As Amazon.com, Amazon.com, Inc.
        company['company_type'] = elems[3].text.lstrip() # Company Type
    elif len(elems) == 3:
        company['founded_date'] = elems[0].text.lstrip() # Founded date
        company['also_known_as'] = elems[1].text.lstrip() #Also Known As Amazon.com, Amazon.com, Inc.
        company['company_type'] = elems[2].text.lstrip() # Company Type
        
    # Legal Name
    try:
        contact = soup.select('div > fields-card > ul > li > field-formatter > blob-formatter > span')[0].text.lstrip()
        if len(contact) == 3:
            company['legal_name'] = elems[0].text.lstrip()
            company['contact_email'] = elems[1].text.lstrip()
            company['contact_phone'] = elems[2].text.lstrip()
        elif len(contact) == 2:
            company['legal_name'] = elems[0].text.lstrip()
            company['contact_phone'] = elems[1].text.lstrip()
    except:
            print("No contact information")
        
    
    # Last Funding Type
    # Number of Exits
    elems = soup.select('ul > li > field-formatter > a')
    for elem in elems:
        if 'last_funding_type' in elem.get('href'):
            company['last_funding_type'] = elem.text.lstrip().rstrip()
        if 'num_exits' in elem.get('href'):
            company['number_of_exits'] = elem.text.lstrip().rstrip()

    # description
    elem = soup.select('body > chrome > div > mat-sidenav-container > mat-sidenav-content > div > ng-component > entity-v2 > page-layout > div > div > div > div > page-centered-layout > div > div > div.main-content > row-card:nth-child(2) > profile-section > section-card > mat-card > div.section-content-wrapper > div > description-card > div > *')
    elems = [e.text.lstrip() for e in elem]
    company['description'] = ' '.join(elems)

    return company

def scrape_data(idx, company_name):
    name = format_name(company_name)

    print_green(f'{idx} Checking {company_name} alias {name}')

    # load the page in "soup" variable
    soup = get_page(f'/organization/{name}')
    if not soup:
        print_green(
            f'{company_name}, alias {name} gave an error while loading')
        with open('../data/error.csv', 'a') as file:
            file.write('\n' + company_name)
        return

    company = company_info(idx, soup)
    crunchbase[urls['company'][idx]] = company

    
def saveFile(start, end, cruchbase):
    filePath = './data/'
    fileNameCSV = 'crunchbase_%03d_%03d.csv'%(start,end)
    pd.DataFrame.from_records(crunchbase).to_csv(filePath+fileNameCSV, index=True)

    fileNameJSON = 'crunchbase_%03d_%03d.json'%(start,end)
    with open(filePath+fileNameJSON, 'w') as fp:
        json.dump(crunchbase, fp, indent=2)
    

for index in range(0,501, 250):
    start = index
    end = index + 250
    companies = list(urls['crunch_url'][start:end])
    for idx, company in enumerate(companies):
        time.sleep(5)        
        scrape_data(idx+start, company)
        
    saveFile(start, end, crunchbase)
    time.sleep(30)

0 Checking walmart alias walmart


js: Failed to set referrer policy: The value 'same-origin' is not one of 'no-referrer', 'no-referrer-when-downgrade', 'origin', 'origin-when-cross-origin', or 'unsafe-url'. The referrer policy has been left unchanged.


1 Checking amazon alias amazon


js: Failed to set referrer policy: The value 'same-origin' is not one of 'no-referrer', 'no-referrer-when-downgrade', 'origin', 'origin-when-cross-origin', or 'unsafe-url'. The referrer policy has been left unchanged.


2 Checking apple alias apple


...

js: Failed to set referrer policy: The value 'same-origin' is not one of 'no-referrer', 'no-referrer-when-downgrade', 'origin', 'origin-when-cross-origin', or 'unsafe-url'. The referrer policy has been left unchanged.


56 Checking goldman-sachs alias goldman-sachs


js: Failed to set referrer policy: The value 'same-origin' is not one of 'no-referrer', 'no-referrer-when-downgrade', 'origin', 'origin-when-cross-origin', or 'unsafe-url'. The referrer policy has been left unchanged.
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...
js: Uncaught SyntaxError: Unexpected token ...


No IPO status
57 Checking raytheon alias raytheon


js: Failed to set referrer policy: The value 'same-origin' is not one of 'no-referrer', 'no-referrer-when-downgrade', 'origin', 'origin-when-cross-origin', or 'unsafe-url'. The referrer policy has been left unchanged.


In [ ]:
print("end")